In [1]:
import os
import warnings
from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
_ = load_dotenv(find_dotenv())
llm = ChatGroq(model="llama3-70b-8192")

### Connection with the db

In [3]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "../data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

### Translate a question into an SQL query

In [12]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": "How many species of trees are in San Francisco?"})
sql_query = response.split("SQLQuery: ")[-1].strip()  # Extract SQL query part
print(sql_query)

In [16]:
### TEST
db.run(sql_query)

'[(148,)]'

In [17]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Executing the SQL query.

In [20]:
def get_query(response):
    return response.split("SQLQuery: ")[-1].strip() 

In [27]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
### Get the sql Query from the question
write_query = create_sql_query_chain(llm, db)

### Run the Query
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | RunnableLambda(get_query) | execute_query

In [28]:
### TEST
response = chain.invoke({"question": "List the species of trees that are present in San Francisco"})

In [29]:
response

'[("Arbutus \'Marina\' :: Hybrid Strawberry Tree",), (\'Afrocarpus gracilior :: Fern Pine\',), ("Thuja occidentalis \'Emerald\' :: Emerald Arborvitae",), ("Magnolia grandiflora \'Little Gem\' :: Little Gem Magnolia",), (\'Platanus x hispanica :: Sycamore: London Plane\',)]'

### Translate the SQL response into a natural language response

In [30]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, 
    corresponding SQL query, and SQL result, 
    answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query = write_query | RunnableLambda(get_query)).assign(result = itemgetter("query") | execute_query)
    | answer_prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke({"question": "List the species of trees that are present in San Francisco"})
print(response)

Based on the SQL result, the species of trees present in San Francisco are:

1. Hybrid Strawberry Tree
2. Fern Pine
3. Emerald Arborvitae
4. Little Gem Magnolia
5. Sycamore: London Plane
